In [1]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# df = pd.read_csv('./Exploration/ixi_demographics.csv')

# #Collect all pre-processed images from .anat directories
# skullstripped_img_list = []
# ori_img_list = []
# seg_img_list = []
# file_id = []
# sample_size = 0
# root_path = "/shared/ixi-dataset/"
# for i in range(19):
#     dir_name = root_path + "IXI-T1-" + str(i)
#     anat_directories = [x for x in os.listdir(dir_name) if x.endswith('.anat')]
#     for anat_directory in anat_directories:
#         ori_file_name = dir_name + '/' + anat_directory + '/T1_biascorr.nii.gz'
#         seg_file_name = dir_name + '/' + anat_directory + "/T1_fast_seg.nii.gz"
#         if anat_directory[0:3] == 'IXI' and os.path.exists(ori_file_name) and os.path.exists(seg_file_name):
#             ori_img = nib.load(ori_file_name).get_data()
#             seg_img = nib.load(seg_file_name).get_data()
#             skullstripped_img = ori_img * (seg_img > 0)
#             ori_img_list.append(ori_img)
#             seg_img_list.append(seg_img)
#             skullstripped_img = (skullstripped_img - np.mean(skullstripped_img)) / np.std(skullstripped_img)
#             skullstripped_img_list.append(skullstripped_img)
#             file_id.append(anat_directory[3:6])
#             sample_size += 1
#             clear_output(wait=True)
#             print(sample_size)

# print(len(skullstripped_img_list))

11


KeyboardInterrupt: 

In [4]:
data = torch.load("../data.pt")
labels = torch.load("labels_13.pt")
print(data.shape,labels.shape)

torch.Size([536, 3, 224, 224]) torch.Size([536, 13])


In [8]:
# df = pd.read_csv('./demographics.csv')
# sheet = df.values
# filtered = list()
# for i in range(0, len(sheet)):
#     if sheet[i, 5][0:3] == 'IXI':
#         filtered.append((sheet[i, 5], sheet[i, 4]))
# sheet = filtered

# #Create the labels for the skullstripped images
# age = list()
# for i in range(0, len(file_id)):
#     for patient in sheet:
#         if patient[0][3:6] == file_id[i]:
#             age.append(patient[1])

# print(len(age))
# onehot_age = list()
# for i in range(0, len(age)):
#     current_age = int(round(age[i]))
#     current_age -= 18
#     one_hot = [0 for i in range(13)]
#     if current_age >= 60:
#         one_hot[-1] = 1
#     else:
#         one_hot[current_age // 5] = 1
#     onehot_age.append(one_hot)
# print(len(onehot_age))

16
16


In [21]:
def train_model(model, optimizer, epochs=1): 
    """Train a model on CIFAR-10 using the PyTorch Module API.
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device) # move the model parameters to CPU/GPU for e in range(epochs):
    # move to device, e.g. GPU
    for t in range(500):
        model.train() # put model to training mode
        x = data
        x = x.to(device=device, dtype=dtype) # move to y = y.to(device=device, dtype=torch.long)device, e.g. GPU
        scores = model(x)
        loss = F.cross_entropy(scores, labels)

        # Zero out all of the gradients for the variables which the optimizer # will update.
        optimizer.zero_grad()
        # This is the backwards pass: compute the gradient of the loss with # respect to each parameter of the model.
        loss.backward()
        # Actually update the parameters of the model using the gradients

        # computed by the backwards pass.
        optimizer.step()
        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item())) 
            # check_accuracy(loader_val, model)
            # print()

In [7]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [10]:
# resnet50 = models.resnet50()
# def resnet50(num_classes=13, pretrained='imagenet'):
#     """Constructs a ResNet-50 model.
#     """
#     model = models.resnet50(pretrained=False)
#     if pretrained is not None:
#         settings = pretrained_settings['resnet50'][pretrained]
#         model = load_pretrained(model, num_classes, settings)
#     model = modify_resnets(model)
#     return model 
# optimizer = optim.SGD(resnet50.parameters(), lr=1e-3,
#                      momentum=0.9, nesterov=True)
# model = resnet50(13)
# train_model(resnet50,optimizer)

AttributeError: 'function' object has no attribute 'parameters'

In [22]:
model = models.resnet50(pretrained=False)
removed = list(model.children())[:-1]
model= torch.nn.Sequential(*removed)
model = torch.nn.Sequential(model, torch.nn.Linear(2048,13))
optimizer = optim.SGD(model.parameters(), lr=1e-3,
                     momentum=0.9, nesterov=True)
train_model(model,optimizer)

RuntimeError: CUDA out of memory. Tried to allocate 1.60 GiB (GPU 0; 11.17 GiB total capacity; 10.09 GiB already allocated; 677.81 MiB free; 121.83 MiB cached)